<a href="https://colab.research.google.com/github/maycmesquia/Houses_Kaggle/blob/main/Houses_vf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Prices - Advanced Regression Techniques

Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=data_description.txt

**Meta**

É sua função prever o preço de venda de cada casa. Para cada Id no conjunto de teste, você deve prever o valor da variável SalePrice. 

**Métrica**

Os envios são avaliados com base no erro quadrático médio (RMSE)  entre o logaritmo do valor previsto e o logaritmo do preço de venda observado. (Tirar logs significa que erros na previsão de casas caras e casas baratas afetarão o resultado igualmente.)

## Preparação de Ambiente


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, normaltest, spearmanr

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Importando Dados

Importando as bases de dados de Treino e Teste

In [9]:
end = '/content/drive/MyDrive/Colab Notebooks/Kaggle - Competicoes/House Prices - Advanced Regression Techniques/'
train = pd.read_csv(end + 'train.csv')
test = pd.read_csv(end + 'test.csv')

train.shape, test.shape

((1460, 81), (1459, 80))

Iremos classificar as variaveis em Qualitativas e Quantitativas.

In [10]:
variaveis_quantitativas = []
variaveis_qualitativas = []

for coluna in train.columns:
    if train[coluna].dtype == 'int64' or train[coluna].dtype == 'float64':
        variaveis_quantitativas.append(coluna)
    else:
        variaveis_qualitativas.append(coluna)

variaveis_quantitativas.remove('SalePrice')
variaveis_quantitativas.remove('Id')

print(variaveis_quantitativas)
print(variaveis_qualitativas)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCon

## Limpeza dos Dados

Iremos substituir os valores vazios (NaN) das variaveis Quantitativas pelas medianas das amostras

Já as variaveis Qualitativas iremos substituir pela moda da amostra.

In [11]:
for variavel in variaveis_quantitativas:
    if train[variavel].isna().sum() > 0:
        train[variavel].fillna(train[variavel].median, inplace=True)

for variavel in variaveis_qualitativas:
    if train[variavel].isna().sum() > 0:
        train[variavel].fillna(train[variavel].mode, inplace=True)

## Verificar Correlação das variaveis

Iremos utilizar o teste de correlação de Spearman, este metodo tem os seguintes pressupostos:

- Uma ou mais variaveis não apresentar distribuição Normal,
- Uma das variaveis pode ou não ser Qualitativas Ordinais

Caso nenhum dos pressupostos existir devemos utilizar Pearson, distribuição Normal!